In [12]:
import os
import json
import numpy as np
from glob import glob
from PIL import Image
from dataclasses import dataclass

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

DATA_ROOT_PATH = "../eye_tracker_auto_labeller/data"

FACE_OVAL_LANDMARK_IDX_LIST = [
    162,  21,  54, 103,  67, 109,  10, 338, 297, 332, 284, 251, 389,
    356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148,
    176, 149, 150, 136, 172,  58, 132,  93, 234, 127, 162,
]
LEFT_EYE_LANDMARK_IDX_LIST = [
    263, 249, 390, 373, 374, 380, 381, 382, 362, 
    398, 384, 385, 386, 387, 388, 466, 263
]
LEFT_IRIS_LANDMARK_IDX_LIST = [475, 476, 477, 474, 475]
RIGHT_EYE_LANDMARK_IDX_LIST = [
     33,   7, 163, 144, 145, 153, 154, 155, 133,
    173, 157, 158, 159, 160, 161, 246, 33
]
RIGHT_IRIS_LANDMARK_IDX_LIST = [471, 472, 469, 470, 471]

OVAL_UPPER_HALF = [361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172,  58, 132]
OVAL_LOWER_HALF = [234, 127, 162,  21,  54, 103,  67, 109,  10, 338, 297, 332, 284, 251, 389, 356, 454]
OVAL_LEFT_HALF =  [338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377]
OVAL_RIGHT_HALF = [148, 176, 149, 150, 136, 172,  58, 132,  93, 234, 127, 162,  21,  54, 103,  67, 109]

class EyeTrackerDataset(Dataset) :
    def __init__(
        self,
        data_root_path = DATA_ROOT_PATH,
        face_oval_landmark_idx_list     = FACE_OVAL_LANDMARK_IDX_LIST,
        left_eye_landmark_idx_list      = LEFT_EYE_LANDMARK_IDX_LIST,
        left_iris_landmark_idx_list     = LEFT_IRIS_LANDMARK_IDX_LIST,
        right_eye_landmark_idx_list     = RIGHT_EYE_LANDMARK_IDX_LIST,
        right_iris_landmark_idx_list    = RIGHT_IRIS_LANDMARK_IDX_LIST,
        return_landmark = True,
        return_image = False
    ) :
        super(EyeTrackerDataset, self).__init__()
        self.DATA_ROOT_PATH = data_root_path
        self.face_oval_landmark_idx_list    = face_oval_landmark_idx_list
        self.left_eye_landmark_idx_list     = left_eye_landmark_idx_list
        self.left_iris_landmark_idx_list    = left_iris_landmark_idx_list
        self.right_eye_landmark_idx_list    = right_eye_landmark_idx_list
        self.right_iris_landmark_idx_list   = right_iris_landmark_idx_list
        self.return_landmark = return_landmark
        self.return_image = return_image

        self.label_path_list = sorted(glob(
            f"{self.DATA_ROOT_PATH}/*/*.json"
        ))
        self.image_path_list = sorted(glob(
            f"{self.DATA_ROOT_PATH}/*/*.png"
        ))

        # there might exists case where eiter image or label does not exist
        # ex) a.json exist but a.png doesn't exist or vice-versa.
        # filter this cases to ensure every data are image-label pair.
        self.label_path_list = list(filter(
            lambda file_name : file_name.replace("json", "png") in self.image_path_list,
            self.label_path_list
        ))
        self.image_path_list = list(filter(
            lambda file_name : file_name.replace("png", "json") in self.label_path_list,
            self.image_path_list
        ))
        assert len(self.label_path_list) == len(self.image_path_list)

        self.to_torch = transforms.ToTensor()

    def __len__(self) :
        return len(self.label_path_list)
    
    def __getitem__(self, index):
        with open(self.label_path_list[index], "r") as fp :
            label_data = json.load(fp)
        mouse_position = torch.Tensor(label_data["mouse_position"])
        data = {
            "mouse_position" : mouse_position
        }
        if self.return_landmark :
            face_landmark_array = torch.Tensor(label_data["face_landmark_array"])
            face_oval_landmark_array  = face_landmark_array[self.face_oval_landmark_idx_list]
            left_eye_landmark_array   = face_landmark_array[self.left_eye_landmark_idx_list]
            right_eye_landmark_array  = face_landmark_array[self.right_eye_landmark_idx_list]
            left_iris_landmark_array  = face_landmark_array[self.left_iris_landmark_idx_list]
            right_iris_landmark_array = face_landmark_array[self.right_iris_landmark_idx_list]

            data["face_landmark_array"] = face_landmark_array
            data["face_oval_landmark_array"] = face_oval_landmark_array
            data["left_eye_landmark_array"] = left_eye_landmark_array
            data["right_eye_landmark_array"] = right_eye_landmark_array
            data["left_iris_landmark_array"] = left_iris_landmark_array
            data["right_iris_landmark_array"] = right_iris_landmark_array
            data["face_oval_mean"] = face_oval_landmark_array.mean(axis=0)


            ver_mean_diff = torch.mean(face_landmark_array[OVAL_LOWER_HALF], axis=0) - torch.mean(face_landmark_array[OVAL_UPPER_HALF], axis=0)
            hor_mean_diff = torch.mean(face_landmark_array[OVAL_RIGHT_HALF], axis=0) - torch.mean(face_landmark_array[OVAL_LEFT_HALF],  axis=0)
            normal_vec = torch.cross(hor_mean_diff, ver_mean_diff)
            data["normal_vec"] = torch.Tensor([
                [0, 0, 0],
                normal_vec.numpy()
            ]) + torch.mean(face_oval_landmark_array, axis=0)

            landmark_array = np.array(label_data["face_landmark_array"])
            ver_mean_diff = np.mean(landmark_array[OVAL_LOWER_HALF], axis=0) - np.mean(landmark_array[OVAL_UPPER_HALF], axis=0)
            hor_mean_diff = np.mean(landmark_array[OVAL_RIGHT_HALF], axis=0) - np.mean(landmark_array[OVAL_LEFT_HALF],  axis=0)
            normal_vec = np.cross(hor_mean_diff, ver_mean_diff)
            normal_vec = np.array([ [0, 0, 0], normal_vec ]) + face_oval_landmark_array.numpy().mean(axis=0)
            
            print(data["normal_vec"])
            print(normal_vec)


        if self.return_image :
            image = Image.open(self.image_path_list[index])
            left_eye_lt_rb = np.array([
                left_eye_landmark_array.numpy().min(axis=0),
                left_eye_landmark_array.numpy().max(axis=0),
            ])[:, :2] * np.array([image.width, image.height])
            #left_eye_lt_rb = left_eye_lt_rb * 2 - left_eye_lt_rb.mean(axis=0)
            left_eye_lt_rb = left_eye_lt_rb.flatten().astype(int)

            right_eye_lt_rb = np.array([
                right_eye_landmark_array.numpy().min(axis=0),
                right_eye_landmark_array.numpy().max(axis=0),
            ])[:, :2] * np.array([image.width, image.height])
            #right_eye_lt_rb = right_eye_lt_rb * 2 - right_eye_lt_rb.mean(axis=0)
            right_eye_lt_rb = right_eye_lt_rb.flatten().astype(int)

            left_eye_image  = self.to_torch(image.crop(left_eye_lt_rb))
            right_eye_image = self.to_torch(image.crop(right_eye_lt_rb))
            
            data["image"] =  self.to_torch(image)
            data["left_eye_image"] =  left_eye_image
            data["right_eye_image"] =  right_eye_image
        
        return data

In [13]:
eye_tracker_dataset = EyeTrackerDataset(DATA_ROOT_PATH)

print(len(eye_tracker_dataset))


data = eye_tracker_dataset[1600]

print(data["normal_vec"])


print(data["face_oval_mean"])

9513
tensor([[0.5209, 0.7386, 0.0452],
        [0.5212, 0.7356, 0.0208]])
[[0.52088994 0.73859817 0.04519788]
 [0.52115936 0.73562229 0.02079852]]
tensor([[0.5209, 0.7386, 0.0452],
        [0.5212, 0.7356, 0.0208]])
tensor([0.5209, 0.7386, 0.0452])


In [1]:
import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

def edge_list_2_path(edge_list) :
    tesel = edge_list
    change_occured = True
    while change_occured :
        change_occured = False
        for idx in range(len(tesel)) :
            target_edge = tesel[idx]
            inner_changed = False
            for e in tesel :
                if e != target_edge and len(e) < 3 :
                    edge = e
                    if target_edge[-1] == edge[0] :
                        target_edge.append(edge[-1])
                        tesel.remove(edge)
                        change_occured = True
                        inner_changed = True
                        break
                    if target_edge[0] == edge[-1] :
                        target_edge.insert(0, edge[0])
                        tesel.remove(edge)
                        change_occured = True
                        inner_changed  = True
                        break
            if inner_changed :
                break
    change_occured = True
    while change_occured :
        change_occured = False
        for idx in range(len(tesel)) :
            source_path = tesel[idx]
            inner_changed = False
            for target_path in tesel :
                if target_path == source_path :
                    continue
                if source_path[-1] == target_path[-1] :
                    target_path.reverse()
                    source_path += target_path[1:]
                    tesel.remove(target_path)
                    change_occured = True
                    inner_changed = True
                    break
                if source_path[0] == target_path[0] :
                    source_path.reverse()
                    target_path += source_path[1:]
                    tesel.remove(source_path)
                    change_occured = True
                    inner_changed = True
                    break
            if inner_changed :
                break
    return tesel


FACE_TESSELATION_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_TESSELATION
)).tolist())
FACE_OVAL_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_FACE_OVAL
)).tolist())
FACE_LIPS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LIPS
)).tolist())
FACE_LEFT_EYEBROW_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_EYEBROW
)).tolist())
FACE_LEFT_EYE_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_EYE
)).tolist())
FACE_LEFT_IRIS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_IRIS
)).tolist())
FACE_RIGHT_EYEBROW_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_EYEBROW
)).tolist())
FACE_RIGHT_EYE_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_EYE
)).tolist())
FACE_RIGHT_IRIS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_IRIS
)).tolist())



objc[18941]: Class CaptureDelegate is implemented in both /opt/anaconda3/envs/first/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1041cc860) and /opt/anaconda3/envs/first/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x169d065a0). One of the two will be used. Which one is undefined.
objc[18941]: Class CVWindow is implemented in both /opt/anaconda3/envs/first/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x102c3ca68) and /opt/anaconda3/envs/first/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x169d065f0). One of the two will be used. Which one is undefined.
objc[18941]: Class CVView is implemented in both /opt/anaconda3/envs/first/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x102c3ca90) and /opt/anaconda3/envs/first/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x169d06618). One of the two will be used. Which one is undefined.
objc[18941]: Class CVSlider is implemented in both /opt/ana

NameError: name 'np' is not defined

In [ ]:
print(FACE_LEFT_EYE_PATH_LIST[0])
print()
print(FACE_LEFT_IRIS_PATH_LIST[0])
print()
print(FACE_RIGHT_EYE_PATH_LIST[0])
print()
print(FACE_RIGHT_IRIS_PATH_LIST[0])

[263, 249, 390, 373, 374, 380, 381, 382, 362, 398, 384, 385, 386, 387, 388, 466, 263]

[475, 476, 477, 474, 475]

[33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246, 33]

[471, 472, 469, 470, 471]


In [5]:
np.arccos( 1 / 2)

1.0471975511965976